In [62]:
using JuMP
using Gurobi
using DataFrames
using CSV
using LinearAlgebra
##### Problem 2 code skeleton

In [ ]:
### Solution

# Note: @__DIR__ points to the directory which contains this script file
# Read data
cd(string(@__DIR__, "/Data"))
sessions = convert(Matrix,CSV.File("Pb2_sessions.csv"; header=true) |> DataFrame!);
courses = convert(Matrix,CSV.File("Pb2_courses.csv"; header=true) |> DataFrame!);
rooms = convert(Matrix,CSV.File("Pb2_rooms.csv"; header=true) |> DataFrame!);
preferences = convert(Matrix,CSV.File("Pb2_preferences.csv"; header=true) |> DataFrame!);
cohorts = convert(Matrix,CSV.File("Pb2_cohorts.csv"; header=true) |> DataFrame!);

cd(@__DIR__)

In [34]:
preferences[1,1]

1

In [ ]:
# Initialize sets and parameters
J = size(sessions,1); #number of sessions
C = size(courses,1); #number of courses
R = size(rooms,1); #number of classrooms
S = size(cohorts,1); #number of student cohorts
D = 2; #number of days
T = 6; #number of time blocks
Q = rooms[:,2]; #classroom capacities
U = courses[:,2]; #course units
N = cohorts[:,2]; #number of students per cohort
ms = cohorts[:,3]; #minimum number of units per cohort
Ms = cohorts[:,4]; #maximum number of units per cohort
lambda = 0.1; #preference penalty

mandatory = zeros(S,C); #mandatory courses per cohort
mandatory[1:6,1:5] .= 1;
mandatory[7:13,6:9] .= 1;
mandatory[14:15,10:11] .= 1;
mandatory[16:17,12:13] .= 1;



In [48]:
# Start building model
timelimit = 10*60;
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", timelimit)

# VARIABLES
@variable(model,z[1:S,1:J,1:D,1:T],Bin);
@variable(model,x[1:J,1:D,1:T,1:R],Bin);

# CONSTRAINTS
@constraint(
    model, [s in 1:S, j in 1:J, d in 1:D, t in 1:T],
    z[s,j,d,t] <= sum(x[j,d,t,r] for r in 1:R)
)

@constraint(
    model,[s in 1:S, d in 1:D, t in 1:T],
    sum(z[s,j,d,t] for j in 1:J) <= 1
)

for c in 1:C
    @constraint(model,
        [s in 1:S],
        sum(z[s,j,d,t]
            for d in 1:D,
                t in 1:T,
                j in sessions[findall(x -> x==c,sessions[:,2]),1])
            <= 1);
end

for s in 1:S
    mandatory_s = findall(x -> x==1,mandatory[s,:]);
    for c in mandatory_s
        @constraint(model,
            sum(z[s,j,d,t]
            for d in 1:D,
                t in 1:T,
                j in sessions[findall(x -> x==c,sessions[:,2]),1])
            >= 1);
    end
end

@constraint(model,[r in 1:R, d in 1:D, t in 1:T],sum(x[j,d,t,r] for j in 1:J) <= 1);

@constraint(model,[j in 1:J],sum(x[j,d,t,r] for r in 1:R, d in 1:D, t in 1:T) == 1);

@constraint(model,[j in 1:J, d in 1:D, t in 1:T],
            sum(N[s]*z[s,j,d,t] for s in 1:S) <= sum(Q[r]*x[j,d,t,r] for r in 1:R))



Academic license - for non-commercial use only - expires 2021-04-27


55×2×6 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},3}:
[:, :, 1] =
 60 z[1,1,1,1] + 60 z[2,1,1,1] + 60 z[3,1,1,1] + 60 z[4,1,1,1] + 60 z[5,1,1,1] + 60 z[6,1,1,1] + 10 z[7,1,1,1] + 10 z[8,1,1,1] + 10 z[9,1,1,1] + 10 z[10,1,1,1] + 10 z[11,1,1,1] + 10 z[12,1,1,1] + 10 z[13,1,1,1] + 5 z[14,1,1,1] + 5 z[15,1,1,1] + 10 z[16,1,1,1] + 10 z[17,1,1,1] + 12 z[18,1,1,1] + 12 z[19,1,1,1] + 12 z[20,1,1,1] + 12 z[21,1,1,1] + 12 z[22,1,1,1] + 12 z[23,1,1,1] + 12 z[24,1,1,1] + 12 z[25,1,1,1] + 12 z[26,1,1,1] + 12 z[27,1,1,1] + 12 z[28,1,1,1] + 12 z[29,1,1,1] + 12 z[30,1,1,1] + 12 z[31,1,1,1] + 12 z[32,1,1,1] + 12 z[33,1,1,1] + 12 z[34,1,1,1] + 12 z[35,1,1,1] + 12 z[36,1,1,1] + 12 z[37,1,1,1] + 12 z[38,1,1,1] + 12 z[39,1,1,1] + 12 z[40,1,1,1] + 12 z[41,1,1,1] + 12 z[42,1,1,1] + 12 z[43,1,1,1] + 12 z[44,1,1,1] + 12 z[45,1,1,1] + 12 z[46,1,1,1] + 12 z[47,1,1,1] + 12 z[48,1,1,1] + 12 z[49,1,1,1] +

In [43]:
#Constraint8: at least ms unit
@constraint(model, [s in 1:S],
        sum(sum(U[c]*z[s,j,d,t]
            for d in 1:D,
                t in 1:T,
                j in sessions[findall(x -> x==c,sessions[:,2]),1]
                ) for c in 1:C)
            >= ms[s]);

In [44]:
#Constraint9: at most Ms unit
@constraint(model, [s in 1:S],
        sum(sum(U[c]*z[s,j,d,t]
            for d in 1:D,
                t in 1:T,
                j in sessions[findall(x -> x==c,sessions[:,2]),1]
                ) for c in 1:C)
            <= Ms[s]);

In [49]:
@objective(model, Max, 
    sum(N[s]*z[s,j,d,t]*(preferences[s,c]+lambda*(1-preferences[s,c])) 
        for d in 1:D, 
            t in 1:T, 
            s in 1:S, 
            c in 1:C, 
            j in sessions[findall(x -> x==c,sessions[:,2]),1])
        )

60 z[1,1,1,1] + 60 z[1,2,1,1] + 60 z[1,3,1,1] + 60 z[1,4,1,1] + 60 z[1,5,1,1] + 60 z[1,6,1,1] + 60 z[1,7,1,1] + 60 z[1,8,1,1] + 60 z[1,9,1,1] + 60 z[1,10,1,1] + 60 z[1,11,1,1] + 60 z[1,12,1,1] + 60 z[1,13,1,1] + 60 z[1,14,1,1] + 60 z[1,15,1,1] + 60 z[1,16,1,1] + 60 z[1,17,1,1] + 60 z[1,18,1,1] + 60 z[1,19,1,1] + 60 z[1,20,1,1] + 60 z[1,21,1,1] + 60 z[1,22,1,1] + 60 z[1,23,1,1] + 60 z[1,24,1,1] + 60 z[1,25,1,1] + 60 z[1,26,1,1] + 60 z[1,27,1,1] + 60 z[1,28,1,1] + 60 z[1,29,1,1] + 60 z[1,30,1,1] + 6 z[1,31,1,1] + 6 z[1,32,1,1] + 60 z[1,33,1,1] + 6 z[1,34,1,1] + 6 z[1,35,1,1] + 6 z[1,36,1,1] + 60 z[1,37,1,1] + 60 z[1,38,1,1] + 6 z[1,39,1,1] + 6 z[1,40,1,1] + 6 z[1,41,1,1] + 6 z[1,42,1,1] + 60 z[1,43,1,1] + 60 z[1,44,1,1] + 6 z[1,45,1,1] + 6 z[1,46,1,1] + 6 z[1,47,1,1] + 60 z[1,48,1,1] + 6 z[1,49,1,1] + 6 z[1,50,1,1] + 6 z[1,51,1,1] + 6 z[1,52,1,1] + 6 z[1,53,1,1] + 60 z[1,54,1,1] + 6 z[1,55,1,1] + 60 z[2,1,1,1] + 60 z[2,2,1,1] + 60 z[2,3,1,1] + 60 z[2,4,1,1] + 60 z[2,5,1,1] + 60 z[2,6,1,1

In [46]:
start = time()
optimize!(model)
solvetime = time() - start

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 41149 rows, 50820 columns and 1020312 nonzeros
Model fingerprint: 0x28cd337d
Variable types: 0 continuous, 50820 integer (50820 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 1757 rows and 1296 columns
Presolve time: 2.42s
Presolved: 39392 rows, 49524 columns, 971334 nonzeros
Variable types: 0 continuous, 49524 integer (49524 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   14015    4.3255384e+03   0.000000e+00   8.980562e+04      5s
   37116    4.8313000e+03   0.000000e+00   0.000000e+00      9s
   37116    4.8313000e+03   0.000000e+00   0.000000e+00      9s
Co

600.6115288734436

In [41]:
objective_value(model)

4746.799999999998

In [47]:
objective_value(model)

4746.799999999997

# Problem 1

In [84]:
"Build shelter location model 2"
function shelter_model_2(distance::Matrix, capacity::Vector, population::Vector)
    # extract problem dimensions from distance matrix and verify coherence of input data
    m, n = size(distance)
    @assert length(capacity) == n
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 30)
    # VARIABLES
    # Whether to open each shelter
    @variable(model, y[1:n], Bin)
    # Whether to serve a particular area from a particular shelter
    @variable(model, x[1:m, 1:n], Bin)
    # CONSTRAINTS
    @constraint(
        model, serve_every_area[i = 1:m],
        sum(x[i, j] for j = 1:n) == 1
    )
    @constraint(
        model, only_serve_from_open_shelter[i = 1:m, j=1:n],
        x[i, j] <= y[j]
    )
    @constraint(
        model, enough_capacity[j=1:n],
        sum(population[i]*x[i, j] for i=1:m) <= capacity[j]
    )
    @constraint(
        model,
        sum(y[j] for j=1:n) <= 10
    )
    # OBJECTIVE
    @objective(
        model, Min, sum(population[i]*distance[i, j] * x[i, j] for i=1:m, j=1:n)
    )
    
    optimize!(model)
    
    return model, objective_value(model), x, y
end

shelter_model_2

In [89]:
"Build shelter location model 2"
function shelter_model_3(distance::Matrix, capacity::Vector, population::Vector)
    # extract problem dimensions from distance matrix and verify coherence of input data
    m, n = size(distance)
    @assert length(capacity) == n
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 30)
    # VARIABLES
    # Whether to open each shelter
    @variable(model, y[1:n], Bin)
    # Whether to serve a particular area from a particular shelter
    @variable(model, x[1:m, 1:n], Bin)
    @variable(model, t>=0)
    # CONSTRAINTS
    @constraint(
        model, serve_every_area[i = 1:m],
        sum(x[i, j] for j = 1:n) == 1
    )
    @constraint(
        model, only_serve_from_open_shelter[i = 1:m, j=1:n],
        x[i, j] <= y[j]
    )
    @constraint(
        model, enough_capacity[j=1:n],
        sum(population[i]*x[i, j] for i=1:m) <= capacity[j]
    )
    @constraint(
        model,
        sum(y[j] for j=1:n) <= 10
    )
    @constraint(
        model, [i=1:m, j=1:n],
        t >= distance[i,j]*x[i,j]
    )
    # OBJECTIVE
    @objective(
        model, Min, t
    )
    
    optimize!(model)
    
    return model, objective_value(model), x, y
end

shelter_model_3

In [58]:
#sessions = convert(Matrix,CSV.File("Pb2_sessions.csv"; header=true) |> DataFrame!);
shelters = convert(Matrix,CSV.File("Pb1_shelters.csv"; header=true) |> DataFrame!);
areas = convert(Matrix,CSV.File("Pb1_areas.csv"; header=true) |> DataFrame!);

#sessions = convert(Matrix,CSV.File("Pb2_sessions.csv"; header=true) |> DataFrame!);
capacities = shelters[:,3];
populations = areas[:,3];

In [77]:
dist = [LinearAlgebra.norm(areas[i, :2] .- shelters[j, :2]) for i=1:200, j=1:55];
@show size(dist);

size(dist) = (200, 55)


In [78]:
dist

200×55 Array{Float64,2}:
 5.81478  2.37045   2.93714    1.27491   …  3.80815   6.21305  5.18368
 1.25041  2.19391   7.5015     5.83927      0.756215  1.64869  0.619321
 4.54193  1.09761   4.20998    2.54775      2.53531   4.94021  3.91084
 6.06639  2.62207   2.68553    1.0233       4.05976   6.46466  5.4353
 5.77301  2.32868   2.97891    1.31668      3.76638   6.17128  5.14191
 6.27009  2.82577   2.48183    0.819599  …  4.26346   6.66836  5.63899
 7.55911  4.11479   1.19281    0.469423     5.55248   7.95738  6.92802
 6.45078  3.00646   2.30114    0.638908     4.44415   6.84905  5.81968
 2.52813  0.916193  6.22379    4.56156      0.5215    2.9264   1.89704
 1.11256  2.33176   7.63936    5.97713      0.894072  1.51083  0.481464
 6.76074  3.31642   1.99118    0.328945  …  4.75411   7.15901  6.12965
 5.34987  1.90555   3.40205    1.73982      3.34324   5.74814  4.71878
 3.84637  0.402045  4.90555    3.24332      1.83974   4.24464  3.21527
 ⋮                                       ⋱         

In [85]:
m, o, x, y = shelter_model_2(dist, capacities, populations)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 11256 rows, 11055 columns and 44055 nonzeros
Model fingerprint: 0xbbc4f594
Variable types: 0 continuous, 11055 integer (11055 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e-01, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Presolve time: 0.07s
Presolved: 11256 rows, 11055 columns, 44055 nonzeros
Variable types: 0 continuous, 11055 integer (11055 binary)
Found heuristic solution: objective 112549.52431

Root relaxation: objective 1.840364e+04, 1487 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 18403.6426    0   58 112549.524 18403.6426  83.6%     -    0s
H 

(A JuMP Model
Minimization problem with:
Variables: 11055
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 200 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 11056 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 11055 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: enough_capacity, only_serve_from_open_shelter, serve_every_area, x, y, 18517.467115412004, VariableRef[x[1,1] x[1,2] … x[1,54] x[1,55]; x[2,1] x[2,2] … x[2,54] x[2,55]; … ; x[199,1] x[199,2] … x[199,54] x[199,55]; x[200,1] x[200,2] … x[200,54] x[200,55]], VariableRef[y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10]  …  y[46], y[47], y[48], y[49], y[50], y[51], y[52], y[53], y[54], y[55]])

In [90]:
m, o, x, y = shelter_model_3(dist, capacities, populations)

Academic license - for non-commercial use only - expires 2021-04-27
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 22256 rows, 11056 columns and 66055 nonzeros
Model fingerprint: 0x9d6ec320
Variable types: 1 continuous, 11055 integer (11055 binary)
Coefficient statistics:
  Matrix range     [6e-04, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 277 rows and 0 columns
Presolve time: 0.20s
Presolved: 21979 rows, 11056 columns, 65554 nonzeros
Variable types: 1 continuous, 11055 integer (11055 binary)

Root relaxation: objective 9.529313e-02, 7467 iterations, 0.47 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.09529    0 2724          -    0.09529      -     -    3s
H    0     

(A JuMP Model
Minimization problem with:
Variables: 11056
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 200 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 11000 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 11056 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 11055 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: enough_capacity, only_serve_from_open_shelter, serve_every_area, t, x, y, 0.5989318805002903, VariableRef[x[1,1] x[1,2] … x[1,54] x[1,55]; x[2,1] x[2,2] … x[2,54] x[2,55]; … ; x[199,1] x[199,2] … x[199,54] x[199,55]; x[200,1] x[200,2] … x[200,54] x[200,55]], VariableRef[y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10]  …  y[46], y[47], y[48], y[49], y

In [86]:
o

18517.467115412004